# ETL 資料轉換

In [5]:
from pathlib import Path
import json
from src.config.constant import RAW_GAME_INFO_SUBFOLDER, MAIN_COLS

In [ ]:
# raw_sub_folder = RAW_GAME_INFO_SUBFOLDER
# root = Path(__file__).resolve().parents[2]
# raw_folder = root / raw_sub_folder

In [ ]:
# 設定路徑及檔案名
raw_folder = Path(r"C:\Users\add41\Documents\Data_Engineer\Project\Steam-Games-Database-with-RAG\data\raw\game_info")
input_file = "game_info_{}.json"

# 設定邊檔案編號並讀取檔案
input_file_num = 1
input_path = raw_folder / input_file.format(input_file_num)

with open(input_path, "r", encoding="utf-8") as f:
    raw_data = json.load(f)


# 取得raw data中的遊戲列表
raw_game_list = raw_data.get("data")

# 僅保留需要的資料欄位
key = list(raw_game_list[0].keys())[0]
raw_game_info = raw_game_list[0].get(key).get("data")
new_game_info = {k: v for k, v in raw_game_info.items() if k in MAIN_COLS}



In [4]:
new_game_info["categories"]

[{'id': 1, 'description': 'Multi-player'},
 {'id': 49, 'description': 'PvP'},
 {'id': 36, 'description': 'Online PvP'},
 {'id': 37, 'description': 'Shared/Split Screen PvP'},
 {'id': 66, 'description': 'Color Alternatives'},
 {'id': 68, 'description': 'Custom Volume Controls'},
 {'id': 75, 'description': 'Keyboard Only Option'},
 {'id': 69, 'description': 'Stereo Sound'},
 {'id': 8, 'description': 'Valve Anti-Cheat enabled'},
 {'id': 62, 'description': 'Family Sharing'}]

In [9]:
new_tag_list = []
for tag in new_game_info["categories"]:
    new_tag_list.append(tag.get("description"))

new_game_info["categories"] = new_tag_list
new_game_info["categories"]

['Multi-player',
 'PvP',
 'Online PvP',
 'Shared/Split Screen PvP',
 'Color Alternatives',
 'Custom Volume Controls',
 'Keyboard Only Option',
 'Stereo Sound',
 'Valve Anti-Cheat enabled',
 'Family Sharing']

In [10]:
new_game_info

{'type': 'game',
 'name': 'Counter-Strike',
 'steam_appid': 10,
 'required_age': 0,
 'is_free': False,
 'detailed_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
 'about_the_game': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
 'short_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic mis